In [1]:
#creating a directory
!mkdir ~/.kaggle

In [2]:
#using credentials to open kaggle dataset
import json
token = {"username":"nishurocks23","key":"26c5d88e7d80c0b16e9c5c7ac10f1c60"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [3]:
!cp /root/.kaggle/kaggle.json ~/.kaggle/kaggle.json

cp: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file


In [4]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
#downloading the zipped dataset folder on cloud
!kaggle datasets download -d crowdflower/twitter-airline-sentiment -p /content

  0% 0.00/2.55M [00:00<?, ?B/s]
100% 2.55M/2.55M [00:00<00:00, 81.8MB/s]


In [7]:
#unzipping the folder
!unzip \*.zip

Archive:  twitter-airline-sentiment.zip
  inflating: Tweets.csv              
  inflating: database.sqlite         


In [8]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 4.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=ee78a0b430bfec54f606cfd5b576fbf9e08348152959bc9267efcfd6df0c3d4c
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=c3120f2b8503a08e72d07a241dc290c55319ac55fe183c76698e8e08aadb6019
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [9]:

#importing libraries for model evaluation and algorithms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import collections
import nltk
from sklearn import preprocessing
from nltk.tokenize import sent_tokenize, word_tokenize
# Packages for data preparation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
#dl libraraies
import keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization,Reshape,Dot,Concatenate,Add,Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
import cv2
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
import os
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
# specifically for deeplearning.
from tensorflow.keras.layers import Dropout, Flatten,Activation,Input,Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
#reading the dataframe
df=pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [11]:
#removing the neutral sentiments considering that positive and negative sentiments matter more 
df=df[df['airline_sentiment']!='neutral']
#now since neutral elements are deleted so I need to reset the indices
df.reset_index(inplace=True,drop=True)
#positive sentiments to 1 and negative to 0 
def partition(x):
    if x =='positive':
        return 1
    return 0
actualSentiment = df['airline_sentiment']
positiveNegative = actualSentiment.map(partition) 
df['Sentiment'] = positiveNegative
df['Sentiment'].value_counts() 

0    9178
1    2363
Name: Sentiment, dtype: int64

In [13]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,Sentiment
0,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),1
1,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0
2,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0
3,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada),0
4,570300616901320704,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada),1


In [12]:
#Setting parameters which will be used throughout
num_words = 15000  # Parameter indicating the number of words we'll put in the dictionary
val_size = 1000  # Size of the validation set
epochs = 20  # Number of epochs we usually start to train with
batch_size = 512  # Size of the batches used in the mini-batch gradient descent
#Taking only two columns since it's a sentiment analysis

In [13]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [14]:
#tweets conssits of every document as an array of tokenized words which are later appended to docs 
tweets=[word_tokenize(tweet) for tweet in df['text']]
docs=[]
for j in range(0,len(tweets)):
    docs.append(tweets[j])

In [15]:
#stops included both the stopwords and punctuations
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
not_list = ["n't", "not", "no"]
stops.update(punctuations)
stops.update(not_list)

In [16]:
#to get the simple pos(part of speech) tag
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [17]:
#to get the pos tag for a word
from nltk import pos_tag
# now we are going to clean our data 
# we will remove stopwords and punctuations and lemmatize each document
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def clean(words):
    output=[]
    for word in words:
        if word.lower() not in stops or word.lower() in not_list:
            pos=pos_tag(word)
            clean_word=lemmatizer.lemmatize(word,pos=get_simple_pos(pos[0][1]))
            output.append(clean_word.lower())
    str1=" ".join(output).encode('utf-8')        
    return str1
docs=[ clean(doc) for doc in docs]      

In [18]:
#docs was the cleaned lemmatized text which has been appended into the dataframe for further use
df['CleanedText']=docs
df['CleanedText']=df['CleanedText'].str.decode("utf-8")
def remove_mentions(input_text):
        return re.sub(r'@\w+', '', input_text)
df.text = df.CleanedText.apply(remove_mentions)
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,Sentiment,CleanedText
0,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,virginamerica plus 've added commercial experi...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),1,virginamerica plus 've added commercial experi...
1,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,virginamerica 's really aggressive blast obnox...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0,virginamerica 's really aggressive blast obnox...
2,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,virginamerica 's really big bad thing,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0,virginamerica 's really big bad thing
3,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,virginamerica seriously would pay 30 flight se...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada),0,virginamerica seriously would pay 30 flight se...
4,570300616901320704,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,virginamerica yes nearly every time fly vx “ e...,NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada),1,virginamerica yes nearly every time fly vx “ e...


In [19]:
#taking only two columns in the dataframe
df=df[['CleanedText','Sentiment']]
df.head()

,CleanedText,Sentiment
0,virginamerica plus 've added commercial experi...,1
1,virginamerica 's really aggressive blast obnox...,0
2,virginamerica 's really big bad thing,0
3,virginamerica seriously would pay 30 flight se...,0
4,virginamerica yes nearly every time fly vx “ e...,1


In [20]:
#taking variables to be used for train test split as X,y
X,Y=df['CleanedText'].values,pd.get_dummies(df.Sentiment.values)

In [21]:
#using tokenizers to create the tokens having no of words=15000(num_words)
tk = Tokenizer(num_words=num_words,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
#Complete data is tokenized to vectors and padding is done using zeros to match its length to the largest text in the dataset.
tk.fit_on_texts(X)
X = tk.texts_to_sequences(X)
X = pad_sequences(X)
#print(X[:2])
print('Fitted tokenizer on {} documents'.format(tk.document_count))
print('{} words in dictionary'.format(tk.num_words))
print('Top 5 most common words are:', collections.Counter(tk.word_counts).most_common(5))

Fitted tokenizer on 11541 documents
15000 words in dictionary
Top 5 most common words are: [('united', 3426), ('flight', 3321), ('usairways', 2651), ('americanair', 2465), ("n't", 1878)]


In [32]:
import pickle
pickle.dump(tk,open('transform2.pkl','wb'))
files.download('transform2.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
#train test split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
print('# Train data samples:', X_train.shape)
print('# Test data samples:', X_test.shape)

# Train data samples: (9232, 27)
# Test data samples: (2309, 27)


In [23]:
#getting validation data as a part of training data
X_train_rest, X_valid, Y_train_rest, Y_valid = train_test_split(X_train,Y_train, test_size=0.1, random_state=37)
print('Shape of validation set:',X_valid.shape)

Shape of validation set: (924, 27)


In [24]:
#Function defined to test the models in the test set
def test_model(model, epoch_stop):
    model.fit(X_test
              , Y_test
              , epochs=epoch_stop
              , batch_size=batch_size
              , verbose=0)
    results = model.evaluate(X_test, Y_test)
    
    return results

In [25]:
embed_dim = 128 #dimension of the word embedding vector for each word in a sequence 
lstm_out = 196  #no of lstm layers
lstm_model = Sequential()
lstm_model.add(Embedding(num_words, embed_dim,input_length = X_train.shape[1]))
#Adding dropout
lstm_model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
#Adding a regularized dense layer
lstm_model.add(layers.Dense(32,kernel_regularizer=regularizers.l2(0.001),activation='relu'))
lstm_model.add(layers.Dropout(0.5))
lstm_model.add(Dense(2,activation='softmax'))
lstm_model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(lstm_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 128)           1920000   
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 32)                6304      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 2,181,170
Trainable params: 2,181,170
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
#model trained on the training data and taking validation data into account to avoid overfitting for 4 epochs 
history_LSTM=lstm_model.fit(X_train_rest, Y_train_rest, epochs = 4, batch_size=batch_size,validation_data=(X_valid, Y_valid),verbose = 1)

Epoch 1/4
17/17 [==============================] - 17s 839ms/step - loss: 0.6674 - accuracy: 0.7378 - val_loss: 0.5032 - val_accuracy: 0.7955
Epoch 2/4
17/17 [==============================] - 13s 790ms/step - loss: 0.4765 - accuracy: 0.7948 - val_loss: 0.3673 - val_accuracy: 0.8377
Epoch 3/4
17/17 [==============================] - 13s 789ms/step - loss: 0.3347 - accuracy: 0.8740 - val_loss: 0.2944 - val_accuracy: 0.8983
Epoch 4/4
17/17 [==============================] - 13s 782ms/step - loss: 0.2267 - accuracy: 0.9429 - val_loss: 0.2332 - val_accuracy: 0.9199


In [27]:
#prediction by our lstm model on the test dataset
lstm_results = test_model(lstm_model, 3)
print('/n')
print('Test accuracy of lstm model: {0:.2f}%'.format(lstm_results[1]*100))

73/73 [==============================] - 1s 19ms/step - loss: 0.1329 - accuracy: 0.9636
/n
Test accuracy of lstm model: 96.36%


In [30]:
lstm_model.save('lstm_model.h5')
from google.colab import files
files.download('lstm_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#building the model using Keras Tuner so taking a range of values for the layers desired to be tuned.
def build_model(hp):  
  tuned_lstm_model = keras.Sequential([     
     # giving range of values for output_dim for tuning for Embedding layer                                  
     keras.layers.Embedding(
        input_dim=num_words, 
        output_dim=hp.Int('embeddings', min_value=32, max_value=128, step=16),
        input_length = X_train.shape[1]
    ),      
      # giving range of values for no. of units for tuning for LSTM layer
      keras.layers.LSTM(
        units=hp.Int('lstm_1_units', min_value=64, max_value=224,step=32), 
        dropout=hp.Int('drop_1_units', min_value=0, max_value=8,step=2)/10,
        recurrent_dropout=0.0
    ),                                                  
    # giving range of values for no. of units for tuning for dense layer
    keras.layers.Dense(
        units=hp.Int('dense_2_units', min_value=32, max_value=128, step=16),
        activation='relu',
        kernel_regularizer=regularizers.l2(0.001)
    ),   
    # giving range of values for rate for tuning for dropout layer
    keras.layers.Dropout(
        rate=hp.Int('drop_1_rate', min_value=1, max_value=8, step=1)/10), 
    keras.layers.Dense(2, activation='softmax')
  ])
  #model compiled using Adam's optimizer with choices for learning rate
  tuned_lstm_model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return tuned_lstm_model

In [ ]:
#A function defined for keras tuner to look at each of the models
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='dir',
    project_name='lstm_sentiment_Analysis')

In [ ]:
#function called to search the results for different combinations of the parameter values in layers
tuner_search.search(X_train_rest,Y_train_rest,epochs=3,validation_data=(X_valid,Y_valid))

Trial 5 Complete [00h 00m 37s]
val_accuracy: 0.9329004287719727

Best val_accuracy So Far: 0.9372294545173645
Total elapsed time: 00h 03m 27s
INFO:tensorflow:Oracle triggered exit


In [ ]:
#to get the best model parameters
tuned_lstm_model=tuner_search.get_best_models(num_models=1)[0]
tuned_lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 48)            720000    
_________________________________________________________________
lstm (LSTM)                  (None, 160)               133760    
_________________________________________________________________
dense (Dense)                (None, 96)                15456     
_________________________________________________________________
dropout (Dropout)            (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 194       
Total params: 869,410
Trainable params: 869,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#model trained on the training data and taking validation data into account to avoid overfitting for 4 epochs 
history_LSTM=lstm_model.fit(X_train_rest, Y_train_rest, epochs = 4, batch_size=batch_size,validation_data=(X_valid, Y_valid),verbose = 1)

Epoch 1/4
17/17 [==============================] - 12s 701ms/step - loss: 0.1209 - accuracy: 0.9693 - val_loss: 0.2269 - val_accuracy: 0.9242
Epoch 2/4
17/17 [==============================] - 12s 694ms/step - loss: 0.1000 - accuracy: 0.9763 - val_loss: 0.2243 - val_accuracy: 0.9264
Epoch 3/4
17/17 [==============================] - 12s 702ms/step - loss: 0.0713 - accuracy: 0.9863 - val_loss: 0.2507 - val_accuracy: 0.9264
Epoch 4/4
17/17 [==============================] - 12s 690ms/step - loss: 0.0548 - accuracy: 0.9901 - val_loss: 0.3078 - val_accuracy: 0.9351


In [ ]:
#prediction by our tuned lstm model on the test dataset. This accuracy is better than the untuned model
lstm_results = test_model(tuned_lstm_model, 5)
print('/n')
print('Test accuracy of lstm model: {0:.2f}%'.format(lstm_results[1]*100))

73/73 [==============================] - 1s 14ms/step - loss: 0.0694 - accuracy: 0.9809
/n
Test accuracy of lstm model: 98.09%


##Now we will see the predictions by our model on sample movie reviews

***Class 1 is positive class and class 0 is negative class***

In [46]:
#First the sample text is a positive review, so lets see its prediction by our model
sample_text = ('The service was good. Everything was so awesome. Highly recommended.')
#converting into sequence because my model is trained on such a sequence
sample = tk.texts_to_sequences([sample_text])
sample = pad_sequences(sample)
sample.shape

(1, 10)

In [47]:
#In training 27 was the no of tokens for each tweet so padding by zeros so that model can accept it
k=np.zeros((1,27))
k[0,-sample.shape[1]:]=sample
k

array([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0., 2423.,
          14., 7534.,   89.,  456., 7534., 2175.,  190., 2243., 3829.]])

In [48]:
#Prediction by model and as expected we get the class as 1 by almost 83%
predictions = lstm_model.predict_classes(np.array(k))
prediction_probability=lstm_model.predict(np.array(k))
print(predictions)
print(prediction_probability[0])

[1]
[0.06591979 0.9340802 ]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
#Prediction by tuned model and as expected we get the class as 1 by almost 95%
predictions = tuned_lstm_model.predict_classes(np.array(k))
prediction_probability=tuned_lstm_model.predict(np.array(k))
print(predictions)
print(prediction_probability[0])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[1]
[0.04985565 0.95014435]


In [31]:
#Secondly the sample text is a negative review, so lets see its prediction by our model
sample_text = ('The movie was pathetic and terrible.Nothing was making any sense. Highly unacceptable.')
#converting into sequence because my model is trained on such a sequence
sample = tk.texts_to_sequences([sample_text])
sample = pad_sequences(sample)
sample.shape

(1, 13)

In [32]:
#In training 27 was the no of tokens for each tweet so padding by zeros so that model can accept it
k=np.zeros((1,27))
k[0,-13:]=sample
k

array([[    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,  2423.,  1348.,
         7534.,   845.,  1778.,   223.,   153.,  7534.,   228., 12070.,
          656.,  2243.,   257.]])

In [33]:
#Prediction by model and as expected we get the class as 0 by 99.99% probability. 
predictions = lstm_model.predict_classes(np.array(k))
prediction_probability=lstm_model.predict(np.array(k))
print(predictions)
print(prediction_probability[0])

[0]
[9.9999213e-01 7.8319226e-06]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
#Prediction by tuned model and as expected we get the class as 0 by 99.99% probability. 
predictions = tuned_lstm_model.predict_classes(np.array(k))
prediction_probability=tuned_lstm_model.predict(np.array(k))
print(predictions)
print(prediction_probability[0])

[0]
[9.999629e-01 3.712654e-05]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
